## Bot Telegram

In [30]:
pip install pyTelegramBotAPI

2921.31s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install pyTelegramBotAPI


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Importación de Paquetes

In [21]:
import telebot
from dotenv import load_dotenv
import os

In [22]:
from telebot.types import InlineKeyboardMarkup
from telebot.types import InlineKeyboardButton

In [23]:
### Instanciamos el bot de telegram
### el único parámetro obligatorio para instanciarlo es el TOKEN.

In [24]:
# importamos nuestro token
load_dotenv()
TELEGRAM_TOKEN = os.getenv('TELEGRAM_TOKEN')
bot_databanda = telebot.TeleBot(TELEGRAM_TOKEN)

## Responder a menasjes que son comandos (básicos)

In [ ]:
# RESPONDER A MENSAJES QUE SON COMANDOS

@bot_databanda.message_handler(commands=['start'])
def cmd_start(message):
    bot_databanda.reply_to(message, "Hola, soy un bot de finanzas. ¿Cómo puedo ayudarte?")

@bot_databanda.message_handler(commands=['fin'])
def cmd_fin(message):
    bot_databanda.reply_to(message, "Nos vemos, recuerda que puedes consultar tu saldo o realizar transferencias.")

@bot_databanda.message_handler(commands=['help'])
def send_help(message):
    bot_databanda.reply_to(message, "Aquí tienes una lista de comandos que puedes usar:\n"
                                     "/start - Iniciar el bot\n"
                                     "/help - Mostrar esta ayuda\n"
                                     "/balance - Consultar tu saldo\n"
                                     "/ingresar - Ingresar monto\n"
                                     "/gasto - Registrar un gasto\n")
                                     

## Funciones más complejas (heavys)

In [ ]:
#que el usuario pueda consultar su saldo si tiene 0 que pueda agregar en base a transferencia

# Diccionario para guardar el saldo de cada usuario
user_saldos = {}
from telebot import types  # Asegúrate de tener esto arriba


@bot_databanda.message_handler(commands=['gasto'])
def cmd_gasto(message):
    bot_databanda.reply_to(message, "💸 Ingresa el monto:")
    bot_databanda.register_next_step_handler(message, process_gasto)

def process_gasto(message):
    try:
        amount = int(message.text)
        if amount <= 0:
            msg = bot_databanda.reply_to(
                message,
                "⚠️ El monto debe ser mayor que 0. Por favor, ingresa un monto válido:"
            )
            bot_databanda.register_next_step_handler(msg, process_gasto)
            return
        
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        markup.add("Sí", "No")
        msg = bot_databanda.send_message(
            message.chat.id,
            f"Has ingresado un monto de ${amount}. ¿Deseas proceder con el ingreso?",
            reply_markup=markup
        )
        bot_databanda.register_next_step_handler(msg, confirm_gasto, amount)

    except ValueError:
        msg = bot_databanda.reply_to(
            message,
            "❌ Ingresa solo números (ejemplo: 1500). Por favor, intenta de nuevo:"
        )
        bot_databanda.register_next_step_handler(msg, process_gasto)

def confirm_gasto(message, amount):
    respuesta = message.text.strip().lower()
    user_id = message.from_user.id
    if respuesta in ["si", "sí"]:
        # Sumar el monto al saldo del usuario
        user_saldos[user_id] = user_saldos.get(user_id, 1000) - amount
        bot_databanda.send_message(message.chat.id, f"✅ Gasto de ${amount} registrado con éxito.")
    elif respuesta == "no":
        bot_databanda.send_message(message.chat.id, "❌ Gasto cancelado.")
    else:
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        markup.add("Sí", "No")
        msg = bot_databanda.send_message(
            message.chat.id,
            "Por favor, responde usando los botones: ¿Deseas proceder con el gasto?",
            reply_markup=markup
        )
        bot_databanda.register_next_step_handler(msg, confirm_gasto, amount)

@bot_databanda.message_handler(commands=['ingresar'])
def cmd_ingresar(message):
    bot_databanda.reply_to(message, "💸 Ingresa el monto:")
    bot_databanda.register_next_step_handler(message, process_amount)

def process_amount(message):
    try:
        amount = int(message.text)
        if amount <= 0:
            msg = bot_databanda.reply_to(
                message,
                "⚠️ El monto debe ser mayor que 0. Por favor, ingresa un monto válido:"
            )
            bot_databanda.register_next_step_handler(msg, process_amount)
            return
        
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        markup.add("Sí", "No")
        msg = bot_databanda.send_message(
            message.chat.id,
            f"Has ingresado un monto de ${amount}. ¿Deseas proceder con el ingreso?",
            reply_markup=markup
        )
        bot_databanda.register_next_step_handler(msg, confirm_ingreso, amount)

    except ValueError:
        msg = bot_databanda.reply_to(
            message,
            "❌ Ingresa solo números (ejemplo: 1500). Por favor, intenta de nuevo:"
        )
        bot_databanda.register_next_step_handler(msg, process_amount)

def confirm_ingreso(message, amount):
    respuesta = message.text.strip().lower()
    user_id = message.from_user.id
    if respuesta in ["si", "sí"]:
        # Sumar el monto al saldo del usuario
        user_saldos[user_id] = user_saldos.get(user_id, 1000) + amount
        bot_databanda.send_message(message.chat.id, f"✅ Ingreso de ${amount} realizada con éxito.")
    elif respuesta == "no":
        bot_databanda.send_message(message.chat.id, "❌ Ingreso cancelado.")
    else:
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        markup.add("Sí", "No")
        msg = bot_databanda.send_message(
            message.chat.id,
            "Por favor, responde usando los botones: ¿Deseas proceder con el ingreso?",
            reply_markup=markup
        )
        bot_databanda.register_next_step_handler(msg, confirm_ingreso, amount)

@bot_databanda.message_handler(commands=['balance'])
def send_balance(message):
    user_id = message.from_user.id
    saldo = user_saldos.get(user_id, 1000)  # Saldo inicial por defecto
    bot_databanda.reply_to(message, f"Tu saldo es de ${saldo}")

In [ ]:
# RESPONDER A MENSAJES QUE SON TEXTO
#@bot_databanda.message_handler(content_types=['text'])

def bot_mensajes_testo(message):
    bot_databanda.reply_to(message.chat.id, "Me enviaste un mensaje de texto: " + message)


In [ ]:
# seteand comandos
bot_databanda.set_my_commands([
    telebot.types.BotCommand("/start", "Iniciar el bot"),
    telebot.types.BotCommand("/help", "Mostrar ayuda"),
    telebot.types.BotCommand("/balance", "Consultar tu saldo"),
    telebot.types.BotCommand("/ingresar", "Realizar un ingreso"),
    telebot.types.BotCommand("/gasto", "Registrar un gasto"),
    telebot.types.BotCommand("/fin", "Finalizar la conversación")
])

True

## Lanzar el programa /bot

In [ ]:
if __name__ == "__main__":
    print("Iniciando el bot...")

    # Seteamos los comandos de nuevo ...
    bot_databanda.set_my_commands([
        telebot.types.BotCommand("/start", "Iniciar el bot"),
        telebot.types.BotCommand("/help", "Mostrar ayuda"),
        telebot.types.BotCommand("/balance", "Consultar tu saldo"),
        telebot.types.BotCommand("/ingresar", "Realizar un ingreso"),
        telebot.types.BotCommand("/gasto", "Registrar un gasto"),
        telebot.types.BotCommand("/fin", "Finalizar la conversación")
    ])

    bot_databanda.infinity_polling() # bucle infinito para recibir mensajes (while true)

    print("Fin = Bot Detenido")

    

Iniciando el bot...


2025-08-20 11:37:23,832 (__init__.py:1121 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2025-08-20 11:37:23,833 (__init__.py:1123 MainThread) ERROR - TeleBot: "Break infinity polling"


Fin = Bot Detenido
